In [1]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install torch==1.7.1+cpu torchvision==0.8.2+cpu torchaudio===0.7.2 -f https://download.pytorch.org/whl/torch_stable.html


In [3]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
# tested in transformers==4.18.0, pytorch==1.7.1
import torch
import transformers
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
# import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
torch.__version__, transformers.__version__

('2.3.0+cu121', '4.40.2')

In [24]:
file_path = 'datasets/100k_gme_reddit_comments_random.csv'

bert_tuning_df = pd.read_csv(file_path)

In [25]:
column_names = bert_tuning_df.columns.tolist()
print(column_names)


['id', 'created_utc', 'body', 'sentiment', 'score', 'merged_body', 'date', 'sentiment_string']


In [26]:
bert_tuning_df = bert_tuning_df.drop(columns=["id", "score", "merged_body", "sentiment_string", 'score'])
# bert_tuning_df['date'] = bert_tuning_df['created_utc'].dt.tz_localize('UTC')

In [27]:
bert_tuning_df.head(10)

,created_utc,body,sentiment,date
0,1635724775,u/all_hail_to_me or he’s trying to say\nMake G...,0.6588,2021-10-31
1,1635724561,Loopring making moves while stock market is cl...,0.0000,2021-10-31
2,1635724440,As it says in Title. \n\nIf you sell your CS D...,0.7876,2021-10-31
3,1635723936,Learned this stuff with GME. I'm a small fry w...,-0.4215,2021-10-31
4,1635723871,Hello and welcome to r/GME!\n\nYou don't have ...,0.9419,2021-10-31
5,1635723394,"Yep. sold all of my company stock and my roth,...",0.0772,2021-10-31
6,1635723339,1)Getting after hours price drop notifications...,-0.7579,2021-10-31
7,1635722971,I feel so safe having my shares of GME in Fide...,0.7133,2021-10-31
8,1635722371,When we get millions per share does that make ...,0.6801,2021-10-31
9,1635722260,Yeah people somehow always forget about things...,0.5574,2021-10-31


In [28]:
# Map sentiment scores from the range [-1, 1] to [0, 1]
bert_tuning_df['sentiment'] = (bert_tuning_df['sentiment'] + 1) / 2



In [29]:
bert_tuning_df['label'] = bert_tuning_df['sentiment'].apply(lambda x: 1 if x >= 0.5 else 0)

In [30]:
column_names = bert_tuning_df.columns.tolist()
print(column_names)

['created_utc', 'body', 'sentiment', 'date', 'label']


In [31]:
df = bert_tuning_df.drop(columns=["sentiment"])

# Now you can work with the DataFrame 'df'
# df = df.head(50000)

In [32]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(81000, 4) (10000, 4) (9000, 4)


In [33]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

/home/az2482/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at yiyanghkust/finbert-pretrain and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['body'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['body'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['body'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

Map:   0%|          | 0/81000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [35]:
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable


In [36]:
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable


In [37]:
pip install transformers==4.30

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 1.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 36.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.7 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
Note: you may need to restart the kernel to use updated packages.


In [38]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.288700,0.263831,0.887778
2,0.208900,0.247506,0.901778
3,0.134200,0.308631,0.902222
4,0.088700,0.408591,0.898667
5,0.053100,0.491763,0.898889


TrainOutput(global_step=12660, training_loss=0.16996536021360664, metrics={'train_runtime': 2927.8589, 'train_samples_per_second': 138.326, 'train_steps_per_second': 4.324, 'total_flos': 2.664023354496e+16, 'train_loss': 0.16996536021360664, 'epoch': 5.0})

In [39]:
model.eval()
trainer.predict(dataset_test).metrics

{'test_loss': 0.3090147376060486,
 'test_accuracy': 0.9009,
 'test_runtime': 22.0901,
 'test_samples_per_second': 452.691,
 'test_steps_per_second': 14.169}

In [40]:
trainer.save_model('models/finbert_fine_tuned_on_100k_datapoints/')